# Classifing last names with character-level RNN

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import random

## Dataset
`https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_train.csv.gz`

`https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_test.csv.gz`

In [10]:
def unpack_dataset():
    ! wget https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_train.csv.gz 
    ! wget https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_test.csv.gz 
    ! mkdir -p data
    ! gunzip names_train.csv.gz 
    ! gunzip names_test.csv.gz
    ! mv names*.csv data

In [11]:
unpack_dataset()

--2020-05-27 15:13:54--  https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_train.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.188.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.188.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50237 (49K) [application/octet-stream]
Saving to: ‘names_train.csv.gz’

names_train.csv.gz  100%[===================>]  49.06K  --.-KB/s    in 0.04s   

2020-05-27 15:13:54 (1.26 MB/s) - ‘names_train.csv.gz’ saved [50237/50237]

--2020-05-27 15:13:54--  https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_test.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.188.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.188.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27541 (27K) [application/octet-stream]
Saving to: ‘names_test.

In [12]:
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/glove.6B.300d.txt'),
 PosixPath('data/glove.6B.100d.txt'),
 PosixPath('data/names_train.csv'),
 PosixPath('data/names_test.csv'),
 PosixPath('data/glove.6B.50d.txt'),
 PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/pmlb'),
 PosixPath('data/quote.tok.gt9.5000'),
 PosixPath('data/glove.6B.200d.txt'),
 PosixPath('data/glove.6B.zip')]

In [13]:
! head data/names_train.csv

### Processing data

In [14]:
df = pd.read_csv(PATH/"names_train.csv", header=None)

In [15]:
# getting a vocabulary of characters
letters = [list(l) for l in df[0].values]
vocab = sorted(list(set(np.concatenate(np.array(letters)))))
vocab[:10]

[' ', "'", ',', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

In [16]:
vocab2id = {key:i for i, key in enumerate(vocab)}
vocab2id[" "] # I am going to use 0 to pad sequences

0

In [17]:
labels = sorted(df[1].unique())
label2id = {key:i for i, key in enumerate(labels)}
label2id

{'Arabic': 0,
 'Chinese': 1,
 'Czech': 2,
 'Dutch': 3,
 'English': 4,
 'French': 5,
 'German': 6,
 'Greek': 7,
 'Irish': 8,
 'Italian': 9,
 'Japanese': 10,
 'Korean': 11,
 'Polish': 12,
 'Portuguese': 13,
 'Russian': 14,
 'Scottish': 15,
 'Spanish': 16,
 'Vietnamese': 17}

In [18]:
def pad_seq(x, seq_len=15, vocab2id=vocab2id):
    x = list(x)
    x = np.array([vocab2id[k] for k in x])
    z = np.zeros(seq_len, dtype=np.int32)
    n = min(seq_len, x.shape[0])
    z[seq_len - n:] = x[0:n]
    return z

In [19]:
x = pad_seq("aabbb")
x

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 29, 29, 30, 30, 30],
      dtype=int32)

In [20]:
# one hot encoding
def seq2matrix(x, vocab_len=55):
    z = np.zeros((x.shape[0], vocab_len))
    z[np.arange(len(x)), x] = 1
    return z

In [21]:
class NameDataset(Dataset):
    def __init__(self, path, vocab2id, label2id, seq_len=15, vocab_len=55):
        self.df = pd.read_csv(path, header=None)
        self.label2id = label2id
        self.vocab2id = vocab2id
        self.seq_len = seq_len
        self.vocab_len = vocab_len 
        self.x = df[0].values
        self.y = [self.label2id[l] for l in df[1].values]
        self.vocab2id = vocab2id
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = pad_seq(self.x[idx], self.seq_len, self.vocab2id)
        x = seq2matrix(x, self.vocab_len)
        return x, self.y[idx]

In [22]:
train = NameDataset(PATH/"names_train.csv", vocab2id, label2id)
val = NameDataset(PATH/"names_test.csv", vocab2id, label2id)

In [23]:
batch_size = 2000
n=len(val)
train_dl = DataLoader(train, batch_size=batch_size)
val_dl = DataLoader(val, batch_size=n)

In [24]:
len(train), len(val)

(13374, 13374)

In [25]:
x,y = train[0]
print(x.shape,y)

(15, 55) 2


## Model

In [26]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()

        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

## Debugging model

In [29]:
vocab_size = 55
hidden_size = 100
n_classes = 18
model = CharRNN(vocab_size, hidden_size, n_classes)

In [30]:
x, y = next(iter(train_dl))

In [31]:
x.shape, y.shape

(torch.Size([2000, 15, 55]), torch.Size([2000]))

In [32]:
batch = x.shape[0]
h = model.initHidden(batch)
x = x.float()
y = y.long()

In [33]:
torch.cat((x[:,0], h), 1).size()

torch.Size([2000, 155])

In [34]:
for ei in range(x.shape[1]):
    y_t, h = model(x[:,ei], h)

In [35]:
# note that just the last x_t is used in the loss
# update
loss = F.cross_entropy(y_t, y)
loss.item()

2.8705217838287354

## Training

In [36]:
vocab_size = 55
hidden_size = 100
n_classes = 18
model = CharRNN(vocab_size, hidden_size, n_classes)

In [37]:
def get_optimizer(model, lr = 0.01, wd = 0.00001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [45]:
def train(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x, y in train_dl:
        batch = x.shape[0]
        h = model.initHidden(batch)
        loss = 0
        x = x.float()
        y = y.long()
        
        for t in range(x.shape[1]):
            out, h = model(x[:,t], h)
        
        loss = F.cross_entropy(out, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

In [46]:
def val_metric(model, val_dl):
    model.eval()
    x, y = next(iter(val_dl))
    x = x.float()
    y = y.long()
    N = x.shape[0]
    h = model.initHidden(N)
    for t in range(x.shape[1]):
        out, h = model(x[:,t], h)
    loss = F.cross_entropy(out, y)
    _, pred = torch.max(out, 1)
    acc = pred.eq(y).sum().float()/N
    return loss.item(), acc.item()

In [47]:
vocab_size = 55
hidden_size = 80
n_classes = 18

In [48]:
def train_loop(model, lr, train_dl, val_dl, epochs=20):
    optim = get_optimizer(model, lr =lr, wd = 0.0)
    for i in range(epochs):
        loss = train(model, optim, train_dl)
        val_loss, val_acc = val_metric(model, val_dl)
        if i%5 == 1: print("train loss %.3f val loss %.3f and val accuracy %.3f" % (loss, val_loss, val_acc))

In [49]:
model = CharRNN(vocab_size, hidden_size, n_classes)

In [50]:
train_loop(model, 0.01, train_dl, val_dl, epochs=20)

train loss 2.093 val loss 1.836 and val accuracy 0.469
train loss 1.749 val loss 1.630 and val accuracy 0.488
train loss 1.497 val loss 1.390 and val accuracy 0.570
train loss 1.332 val loss 1.231 and val accuracy 0.622


In [51]:
train_loop(model, 0.001, train_dl, val_dl, epochs=20)

train loss 1.122 val loss 1.108 and val accuracy 0.665
train loss 1.086 val loss 1.075 and val accuracy 0.681
train loss 1.064 val loss 1.049 and val accuracy 0.687
train loss 1.042 val loss 1.026 and val accuracy 0.692


In [52]:
train_loop(model, 0.001, train_dl, val_dl, epochs=20)

train loss 1.041 val loss 1.023 and val accuracy 0.690
train loss 1.010 val loss 0.998 and val accuracy 0.702
train loss 0.992 val loss 0.977 and val accuracy 0.711
train loss 0.977 val loss 0.961 and val accuracy 0.716


# Model with character embeddings 

In [53]:
class NameDatasetEmb(Dataset):
    def __init__(self, path, vocab2id, label2id, seq_len=15, vocab_len=55):
        self.df = pd.read_csv(path, header=None)
        self.label2id = label2id
        self.vocab2id = vocab2id
        self.seq_len = seq_len
        self.vocab_len = vocab_len 
        self.x = df[0].values
        self.y = [self.label2id[l] for l in df[1].values]
        self.vocab2id = vocab2id
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = pad_seq(self.x[idx], self.seq_len, self.vocab2id)
        return x, self.y[idx]

In [54]:
train_2 = NameDatasetEmb(PATH/"names_train.csv", vocab2id, label2id)
val_2 = NameDatasetEmb(PATH/"names_test.csv", vocab2id, label2id)

In [55]:
batch_size = 2000
n = len(val_2)
train_dl_2 = DataLoader(train_2, batch_size=batch_size)
val_dl_2 = DataLoader(val_2, batch_size=n)

In [56]:
train_2[0]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 32, 47, 37, 48],
       dtype=int32), 2)

In [57]:
class CharEmbRNN(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, output_size):
        super(CharEmbRNN, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(emb_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = x.long()
        x = self.emb(x)
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

## Train 

In [58]:
vocab_size = 55
emb_size = 30
hidden_size = 80
n_classes = 18
model = CharEmbRNN(vocab_size, emb_size, hidden_size, n_classes).cuda()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
train_loop(model, 0.01, train_dl_2, val_dl_2, epochs=50)

In [ ]:
train_loop(model, 0.01, train_dl_2, val_dl_2, epochs=50)

In [ ]:
train_loop(model, 0.01, train_dl_2, val_dl_2, epochs=20)

In [ ]:
train_loop(model, 0.01, train_dl_2, val_dl_2, epochs=20)

In [ ]:
train_loop(model, 0.001, train_dl_2, val_dl_2, epochs=50)

# References
This notebook is a modified version of this tutorial
http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html. Here I implement vanilla RNNs.